In [62]:
import utilities
import matplotlib.pyplot as plt
import numpy as np
import AdaBoost

In [63]:
a = np.array([[1,2,3], [4,3,1]])  # Can be of any shape
indices = np.where(a == a.max())
print(indices[0][0])

1


In [64]:
height = width = 16

In [65]:
train_data = 'Datasets/full_dataset.dat'

In [66]:
data,labels = utilities.read_data(train_data,width,height)

In [67]:
train_data,test_data,train_labels,test_labels = utilities.data_split(data,labels,0.3)
print(train_data.shape)
print(test_data.shape)
print(train_labels.shape)
print(test_labels.shape)

(1446, 256)
(620, 256)
(1446,)
(620,)


In [68]:
data_per_class_dictionary = utilities.collect_each_class_images(train_data,train_labels,10)

287 images of class 0 found
233 images of class 1 found
163 images of class 2 found
104 images of class 3 found
106 images of class 4 found
84 images of class 5 found
116 images of class 6 found
132 images of class 7 found
103 images of class 8 found
118 images of class 9 found


In [69]:
def one_VS_all_training(class_number,BOOSTING_ROUNDS):
    
    models_dict = {}
    for n in range(0,class_number):
        print("Training a classifier for: " + str(n))
        dat,lbl = utilities.create_1_VS_others_dataset(n,data_per_class_dictionary)
        model = AdaBoost.AdaBoost()
        model.fit(dat,lbl,BOOSTING_ROUNDS)
        models_dict[n] = model
        ##model.plot_learning_process()
        
    return models_dict

In [70]:
def one_VS_all_testing(models_dict,data):
    
    models_confidence = []
    for key in models_dict:
        model = models_dict.get(key)
        _,confidence = model.predict(data)
        models_confidence.append(confidence)
        
    predictions = []
        
    for i in range(len(data)):
        results = []
        for j in range(len(models_confidence)):
            results.append((j,models_confidence[j][i]))
            
        ## return as the prediction of the i-th datum the prediction of the classifier with the highest confidence
        predictions.append(sorted(results, key=lambda tup: tup[1],reverse = True)[0][0])
        
    return predictions

In [71]:
pair_datasets = utilities.create_1_VS_1_dataset(data_per_class_dictionary)
print(len(pair_datasets))

45


In [72]:
def one_vs_one_training(pair_datasets):
    
    pairwise_models_dict = {}
    
    for  key in pair_datasets:
        
        (class_1,class_2) = key
        
        print("Training a classifier for pair: " + str(class_1) + " " + str(class_2))
        model = AdaBoost.AdaBoost()
        model.fit(pair_datasets.get(key)[0],pair_datasets.get(key)[1],BOOSTING_ROUNDS)
        
        pairwise_models_dict[key] = model
        
    return pairwise_models_dict

In [73]:
def one_vs_one_testing(pair_models,data,num_classes):
    
    predictions = []
    
    confidence_of_each_classifier = np.zeros((len(data),num_classes))
    for  key in pair_models:
        (class_1,class_2) = key
        model = pair_models.get(key)
        _,confidence = model.predict(data)
        for i in range(len(confidence)):
            if(confidence[i] > 0):
                confidence_of_each_classifier[i][class_1] += abs(confidence[i])
            else:
                confidence_of_each_classifier[i][class_2] += abs(confidence[i])
                
    for result in confidence_of_each_classifier:
        predictions.append(np.where(result == np.amax(result))[0][0])
        
    return predictions

In [74]:
BOOSTING_ROUNDS = 50
class_number = 10
#models_dict = one_VS_all_training(class_number,BOOSTING_ROUNDS)
pair_models = one_vs_one_training(pair_datasets)

Training a classifier for pair: 0 1
Training a classifier for pair: 0 2
Training a classifier for pair: 0 3
Training a classifier for pair: 0 4
Training a classifier for pair: 0 5
Training a classifier for pair: 0 6
Training a classifier for pair: 0 7
Training a classifier for pair: 0 8
Training a classifier for pair: 0 9
Training a classifier for pair: 1 2
Training a classifier for pair: 1 3
Training a classifier for pair: 1 4
Training a classifier for pair: 1 5
Training a classifier for pair: 1 6
Training a classifier for pair: 1 7
Training a classifier for pair: 1 8
Training a classifier for pair: 1 9
Training a classifier for pair: 2 3
Training a classifier for pair: 2 4
Training a classifier for pair: 2 5
Training a classifier for pair: 2 6
Training a classifier for pair: 2 7
Training a classifier for pair: 2 8
Training a classifier for pair: 2 9
Training a classifier for pair: 3 4
Training a classifier for pair: 3 5
Training a classifier for pair: 3 6
Training a classifier for pa

In [75]:
#test_predictions = one_VS_all_testing(models_dict,test_data)
test_predictions = one_vs_one_testing(pair_models,test_data,10)

In [76]:
print(utilities.calculate_accuracy(test_predictions,test_labels))

85.0
